In [3]:
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime


# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'D:/DATASETS/datasets/FaceMaskDataset/train/',
        target_size=(224,224),
        batch_size=32 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'D:/DATASETS/datasets/FaceMaskDataset/test/',
        target_size=(224,224),
        batch_size=32,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=15,
        validation_data=test_set,

        )

model.save('myy_model.h5',model_saved)
#To test for individual images







Found 2520 images belonging to 2 classes.
Found 1295 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
11/79 [===>..........................] - ETA: 1:10 - loss: 0.7183 - accuracy: 0.5455

D:\Anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


79/79 [==============================] - 120s 2s/step - loss: 0.2876 - accuracy: 0.8683 - val_loss: 0.5587 - val_accuracy: 0.8008
Epoch 2/15
79/79 [==============================] - 101s 1s/step - loss: 0.1120 - accuracy: 0.9623 - val_loss: 0.6971 - val_accuracy: 0.7822
Epoch 3/15
79/79 [==============================] - 101s 1s/step - loss: 0.0741 - accuracy: 0.9734 - val_loss: 1.2987 - val_accuracy: 0.6749
Epoch 4/15
79/79 [==============================] - 98s 1s/step - loss: 0.0779 - accuracy: 0.9758 - val_loss: 1.5119 - val_accuracy: 0.6471
Epoch 5/15
79/79 [==============================] - 103s 1s/step - loss: 0.0670 - accuracy: 0.9778 - val_loss: 0.9227 - val_accuracy: 0.7359
Epoch 6/15
79/79 [==============================] - 106s 1s/step - loss: 0.0720 - accuracy: 0.9774 - val_loss: 0.6760 - val_accuracy: 0.7992
Epoch 7/15
79/79 [==============================] - 103s 1s/step - loss: 0.0758 - accuracy: 0.9770 - val_loss: 1.1931 - val_accuracy: 0.6618
Epoch 8/15
79/79 [=======

In [4]:
model.evaluate(test_set,verbose=0)


[1.225259780883789, 0.7629343867301941]

In [ ]:
test_image = cv2.resize(cv2.imread("D:/DATASETS/datasets/FaceMaskDataset/test/without_mask/100.jpg", cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
test_image = np.array(test_image).reshape( -1, IMG_SIZE, IMG_SIZE, 1)
prediction = model.predict({'input': test_image })

In [1]:
#IMPLEMENTING LIVE DETECTION OF FACE MASK
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow.keras
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime

from tensorflow.keras.preprocessing import image
import cv2
mymodel=load_model('myy_model.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(224,224,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()